# 章 データのクリーニングと前処理

In [2]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 欠損値の取り扱い

p209
- デフォルトでは__記述統計量 (describe)__ の算出時にはすべての欠損値が除外される。
- pandasは、欠損値の表現として浮動小数値 __NaN__（非数値、Not a Numberの意味）を使う。
- pandasでは、R言語での慣例にならって、欠損値を __NA__ と呼ぶ。
- NAは利用不可（not available）の意味。
- 統計アプリケーションにおいてNAというデータは、存在しないデータを指す場合と、存在するが観測できなかったデータ（例えば、収集過程で問題が発生したデータなど）を指す。
- 欠損値自体の分析をする必要がしばしばある。
- データの収集時に問題が起きていないかという観点で確認する場合や、欠損値によってデータ内に偏りが発生していないか等、目的はさまざま。
- Python組み込みの値Noneがオブジェクトの配列に含まれている場合、欠損値として扱わる。

In [3]:
# Python組み込みの値Noneがオブジェクトの配列に含まれている場合も、欠損値として扱われる。
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

p211
- 欠損値の確認は、__`dropna`__、__`fillna`__、__`isnull`__、__`notnull`__等が用意されている。（詳細は後述）

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 欠損値を削除する

p211
- シリーズに対して__`dropna`__を用いると、欠損値でないデータとそのインデックスのみを持ったシリーズが戻される。

__シリーズの場合__

In [6]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])

# 削除前
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [7]:
# 削除後
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [8]:
# data.dropna()と同じ
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

__データフレームの場合__

p211
- __`dropna`__メソッドのデフォルトでは、欠損値を1つでも含む行をすべて削除する。
- コピーを作成し、適用元のデータフレームに影響を与えない。

In [9]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [11]:
# dropnaメソッドはコピーを作成する。
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


p212
- how='all'を指定すると、すべてのデータが欠損値である行のみが削除される。
- 列を削除する場合は、axis=1を指定する。

In [12]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [15]:
# 4列目を作成しNAで埋める
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [16]:
# 要素が全てNAである列を削除
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [17]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,-0.204708,0.478943,-0.519439
1,-0.555730,1.965781,1.393406
2,0.092908,0.281746,0.769023
3,1.246435,1.007189,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [18]:
# 1～3行目までの2列目をNAに変更
df.iloc[:4, 1] = NA

# 1～2行目までの3列目をNAに変更
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [19]:
# デフォルトはNAを含む行を全て削除
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


p212
- 一定数の観測値が含まれる行だけを保持したい場合は、__`thresh`__で一定数を指定。

In [23]:
# 行に2つ以上NAがあった場合、削除対象とする。
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### 欠損値を穴埋めする

p213
- __`fillna`__で、欠損値を置き換えることができる。
- 引数にディクショナリを与える事で、列ごとに異なる値で埋めることができる。
- __`fillna`__メソッドはデフォルトでは新しいオブジェクトを戻します。
- __inplace=True__を指定する事で、置き換え元のオブジェクトを直接変更することも可能。

p215
- __`fillna`__の引数は、[表7-2](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html)参照。

In [21]:
# 置き換え前
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [22]:
# NAをゼロで埋める
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [24]:
# ディクショナリを使用して、列ごとに異なる値で埋める
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


1行目が 0.5、2行目がゼロで埋められた。

In [25]:
# inplace=Trueで、置き換え元のオブジェクトを変更
df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


p214
- 再インデックス付けのときと同じ穴埋め方法が使える。

In [26]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [27]:
# 前方（同列の上）の値で穴埋め
df.fillna(method='ffill')

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,0.124121,-2.370232
5,-1.265934,0.124121,-2.370232


In [28]:
# limitでffillで埋める最大数を指定
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


In [29]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [30]:
# NA値を平均で埋める
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

平均値：（1 + 3.5 + 7 ) / 3 = 11.5 /3 = 3.8333333333333335

In [31]:
11.5 / 3

3.8333333333333335

## データの変形

### データの変形

In [32]:
# 要素に重複があるデータを作成
data = pd.DataFrame({
    'k1': ['one', 'two'] * 3 + ['two'],
    'k2': [1, 1, 2, 3, 3, 4, 4]
})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


p216
- __`duplicated`__メソッドは真偽値のシリーズを戻す。

In [34]:
# 重複行の真偽値を返す。（例では、6、7行目が重複）
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

p216
- __`drop_duplicates`__メソッドは重複を削除し、__`duplicated`__の結果が __False__ の要素のみを持つデータフレームを戻す。
- デフォルトではすべての列が同じ値の場合に重複と判定する。
- 列名を指定する事で、重複の検出対象を指定する事も可能。（リスト形式で複数指定可能）
- デフォルトでは、重複が見つかった場合に最初の値を残す。（最初の値以外を重複と判定する。）
- __keep='last'__と指定すると、最後に見つかった値以外を重複と判定する。

In [35]:
# 7行目のみ削除したデータフレームを返却
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [37]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [38]:
# k1列のみ重複の検出対象とする
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [40]:
# 最後に見つかった値以外を重複とみなす。
data.drop_duplicates(['k1'], keep='last')

,k1,k2,v1
4,one,3,4
6,two,4,6


### 関数やマッピングを用いたデータの変換

In [51]:
data = pd.DataFrame({
    'food': [
        'bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon',
        'pastrami', 'honey ham', 'nova lox'
    ],
    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [43]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

In [45]:
# food列の値を小文字に変換して抽出
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [49]:
type(lowercased)

pandas.core.series.Series

__lowercased__と__meat_to_animal__のキー値を紐づけて、__meat_to_animal__の値を__animal__列として追加 
（__`map`__はシリーズのメソッド）

In [52]:
# meat_to_animalの値をanimal列として追加
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


__lamda を使った書き方__

In [54]:
data['animal'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### 値の置き換え

p219
- __`replace`__メソッドによる置き換えがシンプルで柔軟性が高い。
- __inplace=True__を指定すれば、適用元に変換を反映できる。


In [3]:
# 置き換え前
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [8]:
# -999をNAに置き換える
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [7]:
# -999と-1000を NA に変換
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [9]:
# リストのインデックスに対応する値で変換（-999 => NA、 -1000 => 0）
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [10]:
# -999 => NA、 -1000 => 0 に変換する、より分かりやすい方法
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

__replaceは元データに影響を与えない事に注意（反映させるには `inplace=True` が必要）__

In [6]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

__データフレームで試す__

In [3]:
data = pd.DataFrame([[1, 2, 3], [4, 5, 6]])
data

,0,1,2
0,1,2,3
1,4,5,6


In [12]:
data.replace({1: "a", 6: 7})

,0,1,2
0,a,2,3
1,4,5,7


__`where`__を使えば、より柔軟な条件が指定可能。

In [4]:
data[2]

0    3
1    6
Name: 2, dtype: int64

In [7]:
data[2] > 4

0    False
1     True
Name: 2, dtype: bool

In [8]:
# 2列目の要素が 4 より大きければ 8 に変換
data[2].where( data[2] > 4, 8) 

0    8
1    6
Name: 2, dtype: int64

In [24]:
# NG な書き方
data.where( data[2] > 4, 8) 

,0,1,2
0,8,8,8
1,4,5,6


### 軸のインデックスの名前を変更する

p221
- シリーズと同様に、軸のインデックスにもmapメソッドがあり、新たな軸のオブジェクトを生成できる。
- __`map`__メソッドの場合は、適用元のオブジェクトに反映される。
- __`rename`__メソッドを使えば、適用元に反映せず、コピーが作成される。  
- __`rename`__メソッドも、__inplace=True__を指定すれば、適用元に反映できる。

In [26]:
# 検証用のデータフレームを作成
data = pd.DataFrame(
    np.arange(12).reshape((3, 4)),
    index=['Ohio', 'Colorado', 'New York'],
    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [27]:
# 渡された文字の4文字目までを大文字に変換
transform = lambda x: x[:4].upper()

# transform関数で変換されたインデックスオブジェクト 
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

__`map`__メソッドでインデックス名を変換

In [29]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


__`rename`__を使えば、インデックス名を変更したコピーが作成される。

In [9]:
# 検証用のデータフレームを作成
data = pd.DataFrame(
    np.arange(12).reshape((3, 4)),
    index=['Ohio', 'Colorado', 'New York'],
    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [10]:
# 行ラベルを小文字に、列ラベルを大文字に変更
data.rename(index=str.lower, columns=str.upper)

,ONE,TWO,THREE,FOUR
ohio,0,1,2,3
colorado,4,5,6,7
new york,8,9,10,11


In [11]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [45]:
# ディクショナリを使用して変更も可能
data.rename(index={'Ohio': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [46]:
# inplace=True で適用元に反映
data.rename(index={'Ohio': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


### 離散化とビニング
p222
- ある程度連続した値を指定した境界値で区切り「ビン」にグループ化する事を__ビニング__と言う。
- ビニングは__`cut`__関数を使用する。
- __`cut`__は Categoricalオブジェクトを作成する。

In [81]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

__ages の各要素を18歳から25歳、26歳から35歳、36歳から60歳、61歳以上の4つのビンに分類する__  

In [82]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [52]:
# Categoricalオブジェクト
type(cats)

pandas.core.arrays.categorical.Categorical

In [63]:
# リスト表現で分類先を確認
print("20 の分類先：", cats[0], type(cats[0]))
print("25 の分類先：", cats[2], type(cats[2]))
print("61 の分類先：", cats[-4], type(cats[-4]))
print("32 の分類先：", cats[-1], type(cats[0 - 1]))

20 の分類先： (18, 25] <class 'pandas._libs.interval.Interval'>
25 の分類先： (18, 25] <class 'pandas._libs.interval.Interval'>
61 の分類先： (60, 100] <class 'pandas._libs.interval.Interval'>
32 の分類先： (25, 35] <class 'pandas._libs.interval.Interval'>


p222
- Categoricalオブジェクトは、カテゴリのインデックスを示す__codes__属性を持つ。
- Categoricalオブジェクトは、各カテゴリの名前を表す__categories__配列を持つ。

In [64]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [65]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

__closed='right'__は、境界値として「右側を含む」の意味。　(18, 25] は 18 < x <= 25

p223
- __`pd.value_counts()`__で、各ビンに含まれるデータ数が分かる。

In [67]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

p223
- 小括弧 "(" は「境界を含まない」）ことを意味する。
- 大括弧 "[" は「境界を含む」ことを意味する。
- 境界値を含ませるか否かは、right=True/False で指定。（leftオプションはない）
- 境界値は何れかのビンに含まれなければいけない為、両側が __colsed__になる事はない。

In [72]:
cats = pd.cut(ages, [18, 26, 36, 61, 100], right=False)
print(cats[0])

[18, 26)


In [71]:
cats = pd.cut(ages, [18, 26, 36, 61, 100], right=True)
print(cats[0])

(18, 26]


p223
- labelsオプションにリストか配列として渡せば、ビンの名前を設定できる。

In [84]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
cats = pd.cut(ages, bins, labels=group_names)
cats[0]

'Youth'

__(18, 25]__ から __Youth__にビンの名前が変更

p223
- 境界値ではなく、ビンの個数でビニング出来る。
- 個数を指定した場合、データの最小値と最大値を基にして、等間隔に区切ったビンが作成される。
- __precision__オプションで小数点以下の境界値の精度に設定する。

In [105]:
# データを20個用意
data = np.random.rand(20)
print("最小:", data.min())
print("最大:", data.max())

最小: 0.07437830484476893
最大: 0.9354917980336812


In [106]:
# 4個のビンを作成、精度は小数点2位まで
cats = pd.cut(data, 4, precision=2)

# ビン名でソート
pd.value_counts(cats).sort_index()

(0.074, 0.29]    4
(0.29, 0.5]      7
(0.5, 0.72]      2
(0.72, 0.94]     7
dtype: int64

p223
- __`qcut`__は分位点でビニングする。（各ビンが等サイズになる）
- 分位点のパーセンタイルは自分で指定可能（0-1まで）

In [108]:
data = np.random.randn(1000)
pd.DataFrame(data).describe()

,0
count,1000.000000
mean,0.022416
std,1.023078
min,-3.745356
25%,-0.603068
50%,0.009611
75%,0.686379
max,3.260383


In [111]:
# 4分位でビニング
cats = pd.qcut(data, 4) 
pd.value_counts(cats).sort_index()

(-3.746, -0.603]     250
(-0.603, 0.00961]    250
(0.00961, 0.686]     250
(0.686, 3.26]        250
dtype: int64

In [114]:
# 分位点を指定
cats = pd.qcut(
    data, [0, 0.1, 0.5, 0.9, 1.],
    labels=["0-10%", "10-50%", "50-90%", "90-100%"])
pd.value_counts(cats).sort_index()

0-10%      100
10-50%     400
50-90%     400
90-100%    100
dtype: int64

p224
- __cutとqcutは、分位点やグループ化を用いた分析をする際にとても便利。__

### 外れ値の検出と除去

In [152]:
# ダミーデータを作成
np.random.seed(12345)

data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


__例）絶対値が3より大きなものを外れ値とする。__  

__列に対して外れ値を確認__

In [153]:
# 3列目（シリーズ）を取得
col = data[2]

# 3列目の要素で絶対値が3より大きな値を抽出
col[np.abs(col) > 3]

5      3.248944
102    3.176873
324    3.260383
499   -3.056990
586   -3.184377
Name: 2, dtype: float64

__行に対して外れ値を確認__  
__`any`__メソッドで列方向で１つでも条件を満たせば Trueがあれば、それ以外は False を返す。（axis=1で行方向)

In [154]:
# 絶対値が、3より大きい値を1つ以上持つすべての行を抽出
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [158]:
# 絶対値が 3 よりも大きい要素に対して、正の場合3、負の場合-3、ゼロの場合は0を返す。 
data[np.abs(data) > 3] = np.sign(data) * 3

# 置換されている事の確認
data[(np.abs(data) == 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.000000,-1.021228
97,-0.774363,0.552936,0.106061,3.000000
102,-0.655054,-0.565230,3.000000,0.959533
305,-2.315555,0.457246,-0.025907,-3.000000
324,0.050188,1.951312,3.000000,0.963301
400,0.146326,0.508391,-0.196713,-3.000000
499,-0.293333,-0.242459,-3.000000,1.918403
523,-3.000000,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.000000,1.369891
808,-0.362528,-3.000000,1.553205,-2.186301


In [157]:
# 最小、最大値の絶対値が 3 を超える事はないはず
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### 順列（ランダムな並べ替え）やランダムサンプリング

p225
- __`numpy.random.permutation`__でデータフレームや、シリーズをランダムに並べ替えらえれる。
- 並べ替えたい軸の長さを引数として与えれば、新しい順序のインデックスを表す整数の配列が得られる。
- __`permutation`__で得られた配列を__`take`__の引数に渡すことで入れ替える事が出来る。（コピーが作成される）

In [160]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [161]:
# インデックスをランダムに入れ替え
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

In [163]:
# インデックスの入れ替えを適用
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


__3行まで並べ替える__

In [165]:
sampler2 = np.random.permutation(3)
sampler2
df.take(sampler2)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11


p227
- ランダムに非復元抽出を行うには__`sample`__を使用する。（引数 n で抽出個数）
- 復元抽出を許す場合は__`replace=True`__を指定する。

In [166]:
# 3行、非復元抽出
df.sample(n=3)

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
0,0,1,2,3


In [169]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

0    5
1    7
2   -1
2   -1
3    6
2   -1
1    7
2   -1
0    5
4    4
dtype: int64

In [168]:
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

### 標識変数やダミー変数の計算

p228
- カテゴリ変数から「ダミー変数」や「標識変数」の行列への変換には__`get_dummies`__を使用する。
- デフォルトではカテゴリー値が列ラベルに使用される。
- 列ラベルにプレフィックスを付けたい場合は、__`prefix`__オプションで指定する。

In [170]:
# key列がカテゴリー変数
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


__key列をダミー変数化__

In [171]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


__プレフィックスに「key」を付与してダミー変数化__

In [173]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


__ダミー変数化したデータを data1 列と結合__

In [174]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


__例）同一要素内に複数のカテゴリがある場合__

In [176]:
# 検証データ
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(
    'datasets/movielens/movies.dat', sep='::', header=None, names=mnames)
movies[:10]

c:\usr\local\bin\eclipse_python\python\3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


__genres列に複数のジャンルが '|' で区切られている。__  
まず、全てのジャンルを洗い出す。

In [186]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))

# 重複ジャンルを省く
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

__ジャンルを列名に使用したゼロ行列を作成__

In [180]:
# ゼロ行列を作成
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies[:3]

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


__ゼロ行列の該当する列名（ジャンル）の箇所を１を代入する。__

In [190]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int32)

In [182]:
# i:movieの行インデックス、gen:movies.genresの値
for i, gen in enumerate(movies.genres):

    # ゼロ行列（dummies）の 列名のインデックス（リスト）を取得   
    indices = dummies.columns.get_indexer(gen.split('|'))
    
    # ゼロ行列のi行の indices列に１を代入する    
    dummies.iloc[i, indices] = 1

In [204]:
# 3行、10列分だけ確認用に表示
dummies.iloc[:3, :10]

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [206]:
# 元データフレーム（movies）とジャンルのダミー変数化行列を結合
movies_windic = movies.join(dummies.add_prefix('Genre_'))

# 確認用に3行、１2列分だけ表示
movies_windic.iloc[:3, :12]

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,Genre_Action,Genre_Crime
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


p230
- パフォーマンス的にNumPyの配列に直接書き込む低レベルの関数を作成して、その処理結果をデータフレーム化する方が良い。

p231
- cutなどの離散化関数とget_dummiesを組み合わせる方法は統計処理において実用的。

In [211]:
np.random.seed(12345)
values = np.random.rand(10)

# 分位点をカテゴリー値とダミー変数
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 文字列操作


### String Object Methods

In [ ]:
val = 'a,b,  guido'
val.split(',')

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

In [ ]:
'::'.join(pieces)

In [ ]:
'guido' in val
val.index(',')
val.find(':')

In [ ]:
val.index(':')

In [ ]:
val.count(',')

In [ ]:
val.replace(',', '::')
val.replace(',', '')

### Regular Expressions

In [ ]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

In [ ]:
regex = re.compile('\s+')
regex.split(text)

In [ ]:
regex.findall(text)

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

In [ ]:
m = regex.search(text)
m
text[m.start():m.end()]

In [ ]:
print(regex.match(text))

In [ ]:
print(regex.sub('REDACTED', text))

In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

In [ ]:
regex.findall(text)

In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

### Vectorized String Functions in pandas

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

In [ ]:
data.str.contains('gmail')

In [ ]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str.get(1)
matches.str[0]

In [ ]:
data.str[:5]

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion